# Simulate a cluster
By its nature, testing and experimenting with distributed software requires multiple machines. Luckily, we are living in the age of virtual machines and containers - we can simulate as many machines we like by using various vm/container technologies. Let's try docker

### Docker
In this section, we will use Docker to simulatre a cluster. Please note that this is not a Docker tutorial, which is why we will avoid using `docker compose`, `docker swarm` or even `Dockerfile`. We are pretending that these are full-fledged machines and a specific vm/container technology. 

First, let's make sure we have docker installed. If not, please install Docker Desktop: https://www.docker.com/products/docker-desktop/

In [3]:
!docker --version

Docker version 27.5.1, build 9f9e405


#### Pull an image

We will set up a cluster of 5 machines. Le's use the official Python docker image: https://hub.docker.com/_/python
(may take around 5 minutes)

In [6]:
%%time
!docker pull continuumio/miniconda3

Using default tag: latest
latest: Pulling from continuumio/miniconda3
Digest: sha256:0c1494093f919a36ba4cf543abf5e2e426deb3e8aa61c9d4074bcc267d264fe5
Status: Image is up to date for continuumio/miniconda3:latest
docker.io/continuumio/miniconda3:latest
CPU times: total: 46.9 ms
Wall time: 1.31 s


#### Set up a network

This will allow our "machines" to talk to each other

In [91]:
!docker network create simulated-cluster

Error response from daemon: network with name simulated-cluster already exists


#### Start a machine which belongs to our simulated cluster

Note that you can get help for docker flags via `!docker run --help`

Here are the flags we are using below:
- `run` This will run an instance of the image we downloaded earlier. Think of the image download as the CD which contains the operating sysstem and `docker run` as us setting up a physical machine to run the operating system
- `-d` This will run the image in the background, so it doesn't take over JupyterHUB (similar to `python ./app.py &`)
- `-i` This will run the image in an interactive manner - so we can connect to it
- `-t` This will run the image via the terminal
- `--rm` remove the image once we are done with it

In [12]:
!docker run -dit --rm --network simulated-cluster --name node1 continuumio/miniconda3

319d09007e3f522ffe4a6790f73cb55d76b8ad796122877eac4527800824f6c2


Once a container is running, you can "ssh" into it via the exec command. This will not work from Jupyter, you should do it from the command line (terminal)

`docker exec -it node1 bash`

Check docker

In [15]:
!docker ps

CONTAINER ID   IMAGE                    COMMAND       CREATED        STATUS                  PORTS     NAMES
319d09007e3f   continuumio/miniconda3   "/bin/bash"   1 second ago   Up Less than a second             node1


Run a few more machines

In [17]:
!docker run -dit --rm --network simulated-cluster --name node2 continuumio/miniconda3
!docker run -dit --rm --network simulated-cluster --name node3 continuumio/miniconda3

d6040cf9bf45fea9087f65c4884d4cebacde771505e20eb09b969edf0fde41d3
f8b796d0bf269354d3b52c0ec95b3b46745614960f4566d4578045dce1002da7


In [18]:
!docker ps

CONTAINER ID   IMAGE                    COMMAND       CREATED         STATUS                  PORTS     NAMES
f8b796d0bf26   continuumio/miniconda3   "/bin/bash"   1 second ago    Up Less than a second             node3
d6040cf9bf45   continuumio/miniconda3   "/bin/bash"   2 seconds ago   Up 1 second                       node2
319d09007e3f   continuumio/miniconda3   "/bin/bash"   3 seconds ago   Up 3 seconds                      node1


#### Install required software in those machines
Notice that these machines don't even have the ping command installed. Let's install it to confirm these machines can talk to each other

In [20]:
!docker exec node1 ping -c 3cnn.com

OCI runtime exec failed: exec failed: unable to start container process: exec: "ping": executable file not found in $PATH: unknown


Even `ping` is not installed??

In [47]:
!docker exec node1 apt-get update 
!docker exec node1 apt-get install -y iputils-ping iproute2

Hit:1 http://deb.debian.org/debian bookworm InRelease
Hit:2 http://deb.debian.org/debian bookworm-updates InRelease
Hit:3 http://deb.debian.org/debian-security bookworm-security InRelease
Reading package lists...
Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  libatm1 libbpf1 libcap2-bin libelf1 libmnl0 libpam-cap libxtables12
Suggested packages:
  iproute2-doc
The following NEW packages will be installed:
  iproute2 iputils-ping libatm1 libbpf1 libcap2-bin libelf1 libmnl0 libpam-cap
  libxtables12
0 upgraded, 9 newly installed, 0 to remove and 3 not upgraded.
Need to get 1573 kB of archives.
After this operation, 5627 kB of additional disk space will be used.
Get:1 http://deb.debian.org/debian bookworm/main amd64 libelf1 amd64 0.188-2.1 [174 kB]
Get:2 http://deb.debian.org/debian bookworm/main amd64 libbpf1 amd64 1:1.1.0-1 [145 kB]
Get:3 http://deb.debian.org/debian bookworm/main amd64 libmnl0 amd

debconf: delaying package configuration, since apt-utils is not installed


In [49]:
!docker exec node1 ping -c 3 cnn.com

PING cnn.com (151.101.195.5) 56(84) bytes of data.
64 bytes from 151.101.195.5 (151.101.195.5): icmp_seq=1 ttl=63 time=80.0 ms
64 bytes from 151.101.195.5 (151.101.195.5): icmp_seq=2 ttl=63 time=29.4 ms
64 bytes from 151.101.195.5 (151.101.195.5): icmp_seq=3 ttl=63 time=33.1 ms

--- cnn.com ping statistics ---
3 packets transmitted, 3 received, 0% packet loss, time 2003ms
rtt min/avg/max/mdev = 29.443/47.499/79.975/23.011 ms


### Can these machines see each other?

In [52]:
!docker exec node2 hostname

d6040cf9bf45


In [54]:
!docker exec node1 ping -c 3 d6040cf9bf45

PING d6040cf9bf45 (172.18.0.3) 56(84) bytes of data.
64 bytes from node2.simulated-cluster (172.18.0.3): icmp_seq=1 ttl=64 time=0.285 ms
64 bytes from node2.simulated-cluster (172.18.0.3): icmp_seq=2 ttl=64 time=0.177 ms
64 bytes from node2.simulated-cluster (172.18.0.3): icmp_seq=3 ttl=64 time=0.076 ms

--- d6040cf9bf45 ping statistics ---
3 packets transmitted, 3 received, 0% packet loss, time 2042ms
rtt min/avg/max/mdev = 0.076/0.179/0.285/0.085 ms


Nice!

### A drive can be shared among containers to avoid donwloading and installing packages multiple times

Notice that if we need to install the same package in each container, those containers will pull packages off the web again and again and again!
We can solve this by creating a shared volume

#### Shared `apt` volume

In [93]:
!docker volume create apt-cache

!docker run -dit --rm --name apt-container-1 --network simulated-cluster -v apt-cache:/var/cache/apt/archives continuumio/miniconda3
!docker run -dit --rm --name apt-container-2 --network simulated-cluster -v apt-cache:/var/cache/apt/archives continuumio/miniconda3
!docker run -dit --rm --name apt-container-3 --network simulated-cluster -v apt-cache:/var/cache/apt/archives continuumio/miniconda3

apt-cache
31af0861fb32b5898858e19671b462b7c36258efbe5b8e8bd539b6a84b783950
25e966b402685541e809e3ff9d78a770714a4a274c56dbfdfe8ce1f611ae9648
883c97ee721d6917de3a2750cdc15f6338793dde0d0506322804a74328bf9354


Now let's see how long it takes to install a package

In [97]:
%%time
!docker exec apt-container-1 sh -c "apt update && apt-get install -y iputils-ping iproute2"

Hit:1 http://deb.debian.org/debian bookworm InRelease
Hit:2 http://deb.debian.org/debian bookworm-updates InRelease
Hit:3 http://deb.debian.org/debian-security bookworm-security InRelease
Reading package lists...
Building dependency tree...
Reading state information...
3 packages can be upgraded. Run 'apt list --upgradable' to see them.
Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  libatm1 libbpf1 libcap2-bin libelf1 libmnl0 libpam-cap libxtables12
Suggested packages:
  iproute2-doc
The following NEW packages will be installed:
  iproute2 iputils-ping libatm1 libbpf1 libcap2-bin libelf1 libmnl0 libpam-cap
  libxtables12
0 upgraded, 9 newly installed, 0 to remove and 3 not upgraded.
Need to get 1573 kB of archives.
After this operation, 5627 kB of additional disk space will be used.
Get:1 http://deb.debian.org/debian bookworm/main amd64 libelf1 amd64 0.188-2.1 [174 kB]
Get:2 http://deb.debian.org/



debconf: delaying package configuration, since apt-utils is not installed


In [99]:
%%time
!docker exec apt-container-2 sh -c "apt update && apt-get install -y iputils-ping iproute2"

Get:1 http://deb.debian.org/debian bookworm InRelease [151 kB]CPU times: total: 78.1 ms
Wall time: 11.6 s

Get:2 http://deb.debian.org/debian bookworm-updates InRelease [55.4 kB]
Get:3 http://deb.debian.org/debian-security bookworm-security InRelease [48.0 kB]
Get:4 http://deb.debian.org/debian bookworm/main amd64 Packages [8792 kB]
Get:5 http://deb.debian.org/debian bookworm-updates/main amd64 Packages [13.5 kB]
Get:6 http://deb.debian.org/debian-security bookworm-security/main amd64 Packages [246 kB]
Fetched 9306 kB in 7s (1353 kB/s)
Reading package lists...
Building dependency tree...
Reading state information...
3 packages can be upgraded. Run 'apt list --upgradable' to see them.
Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  libatm1 libbpf1 libcap2-bin libelf1 libmnl0 libpam-cap libxtables12
Suggested packages:
  iproute2-doc
The following NEW packages will be installed:
  iproute2 iputils-pi



debconf: delaying package configuration, since apt-utils is not installed
